[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ClimateMatchAcademy/course-content/blob/main/tutorials/W1D5_ClimateModeling/student/W1D5_Tutorial2.ipynb) &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/tutorials/W1D5_ClimateModeling/student/W1D5_Tutorial2.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

# Tutorial 2 : Energy Balance


**Week 1, Day 5, Climate Modeling**

**Content creators:** Jenna Pearson

**Content reviewers:** Dionessa Biton, Younkap Nina Duplex, Zahra Khodakaramimaghsoud, Will Gregory, Peter Ohue, Derick Temfack, Yunlong Xu, Peizhen Yang, Chi Zhang, Ohad Zivan

**Content editors:** Brodie Pearson, Abigail Bodner, Ohad Zivan, Chi Zhang

**Production editors:** Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2023 Sponsors:** NASA TOPS and Google DeepMind


# Tutorial Objectives

In this tutorial students will learn about the components that define energy balance, including insolation and albedo.

By the end of this tutorial students will be able to:

- Calculate the albedo of Earth based on observations.
- Define and find the equilibrium temperature under the assumption of energy balance.
- Understand the relationship between transmissivity and equilibrium temperature.


# Setup

In [ ]:
# imports
import xarray as xr  # used to manipulate data and open datasets
import numpy as np  # used for algebra and array operations
import matplotlib.pyplot as plt  # used for plotting

In [ ]:
# @title Figure settings
import ipywidgets as widgets  # interactive display

%config InlineBackend.figure_format = 'retina'
plt.style.use(
    "https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle"
)

In [ ]:
# @title Video 1: Energy Balance

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'MOo5Xa-Nfzc'), ('Bilibili', 'BV1oV4y127uo')]
tab_contents = display_videos(video_ids, W=730, H=410)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

In [ ]:
# @markdown
from ipywidgets import widgets
from IPython.display import IFrame

link_id = "ce6mx"

download_link = f"https://osf.io/download/{link_id}/"
render_link = f"https://mfr.ca-1.osf.io/render?url=https://osf.io/{link_id}/?direct%26mode=render%26action=download%26mode=render"
# @markdown
out = widgets.Output()
with out:
    print(f"If you want to download the slides: {download_link}")
    display(IFrame(src=f"{render_link}", width=730, height=410))
display(out)

# Section 1 : A Radiating Sun

## Section 1.1: Incoming Solar Radiation (Insolation) and Albedo ($\alpha$)

Just as Earth emits radiation, so does the sun. The incoming solar radiation, called **[insolation](https://glossary.ametsoc.org/wiki/Insolation)**. From the 'All Sky' Energy budget shown below, this is observed to be $Q = 340 W m^{-2}$.

Some of this radiation is reflected back to space (for example off of ice and snow or clouds).

From the 'All Sky' energy budget below, the amount reflected back is $F_{ref} = 100 W m^{-2}$.

![Global Mean Energy Budget](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_7_2.png)
Figure 7.2 | Schematic representation of the global mean energy budget of the Earth (upper panel), and its equivalent without considerations of cloud effects (lower panel). Numbers indicate best estimates for the magnitudes of the globally averaged energy balance components in W m–2 together with their uncertainty ranges in parentheses (5–95% confidence range), representing climate conditions at the beginning of the 21st century. Note that the cloud-free energy budget shown in the lower panel is not the one that Earth would achieve in equilibrium when no clouds could form. It rather represents the global mean fluxes as determined solely by removing the clouds but otherwise retaining the entire atmospheric structure. This enables the quantification of the effects of clouds on the Earth energy budget and corresponds to the way clear-sky fluxes are calculated in climate models. Thus, the cloud-free energy budget is not closed and therefore the sensible and latent heat fluxes are not quantified in the lower panel. Figure adapted from Wild et al. (2015, 2019). (Credit: [IPCC AR6 Report](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_7_2.png))


The _fraction_ of reflected radiation is captured by the **albedo (**$\mathbf{\alpha}$**)**

\begin{align}
\alpha = \frac{F\_{ref}}{Q}
\end{align}

Albedo is a unitless number between 0 and 1. We can use this formula to find the albedo of Earth.


In [ ]:
# define the observed insolation based on observations from the IPCC AR6 Figure 7.2
Q = 340  # W m^-2

# define the observed reflected radiation based on observations from the IPCC AR6 Figure 7.2
F_ref = 100  # W m^-2

# plug into equation
alpha = F_ref / Q  # unitless number between 0 and 1

# display answer
print("Albedo: ", alpha)

### Questions 1.1: Climate Connection

1. Keeping insolation ($Q$) constant, what does a low albedo imply? What about a high albedo?
2. There are two components to albedo, the reflected radiation in the numerator and the insolation in the denomenator. Do you think one or both of these have changed over Earth's history?


## Section 1.2 : Absorbed Shortwave Radiation (ASR)

The **absorbed shortwave radiation (ASR)** is the amount of this isolation that is _not_ reflected, and actually makes it to Earth's surface. Thus,

\begin{align}
ASR = Q-F\_{ref} = (1-\alpha)Q
\end{align}

From observations, we can esimate the absorbed shortwave radiation.


In [ ]:
# plug into equation
ASR = (1 - alpha) * Q

# display answer
print("Absorbed Shortwave Radiation: ", ASR, " W m^-2")

### Questions 1.2: Climate Connection

1. Compare the value of ASR to the observed OLR of $239 W m^{-2}$. Is it more or less? What do you think this means?
2. Does this model take into account any effects of gases in that atmosphere on the incoming shortwave radiation that makes it to Earth's surface? Are there any greenhouse gases you think are important and should be included in more complex models?

# Section 2 : Energy Balance

## Section 2.1: Equilibrium Temperature

Energy Balance is achieved when radiation absorbed by Earth's surface (ASR) is equal to longwave radiation going out to space (OLR). That is

\begin{align}
ASR = OLR
\end{align}

By substituting into the equations from previous sections, we can find the surface temperature of Earth needed to maintain this balance. This is called the **equilibrium temperature (** $\mathbf{T_{eq}}$ **)**.

Recall $OLR = \tau\sigma T^4$ and $ASR = (1-\alpha)Q$. The **equilibrium temperature** is the temperature the system would have if energy balance was perfectly reached. Assuming energy balance, we will call the emission temperature denoted previously the equilibrium temperature ($T_{eq}$) instead. Thus,

\begin{align}
(1-\alpha)Q = ASR = OLR = \tau\sigma T\_{eq}^4
\end{align}

Solving for $T_{eq}$ we find

\begin{align}
T\_{eq} = \sqrt[4]{\frac{(1-\alpha)Q}{\tau\sigma}}
\end{align}

Let's calculate what this should be for Earth using observations:


In [ ]:
# define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
sigma = 5.67e-8  # W m^-2 K^-4

# define transmissivity (calculated previously from observations in tutorial 1)
tau = 0.6127  # unitless number between 0 and 1

# plug into equation
T_eq = (((1 - alpha) * Q) / (tau * sigma)) ** (1 / 4)

# display answer
print("Equilibrium Temperature: ", T_eq, "K or", T_eq - 273, "C")

# Section 3 : Climate Change Scenario

## Section 3.1: Increasing Greenhouse Gas Concentrations

Assume due to the increasing presence of greenhouse gases in the the atmosphere, that $\tau$ decreases to $0.57$.

We can then use our climate model and python to find the new equilibrium temperature.


In [ ]:
# define transmissivity (assupmtion in this case)
tau_2 = 0.57  # unitless number between 0 and 1

# plug into equation
T_eq_2 = (((1 - alpha) * Q) / (tau_2 * sigma)) ** (1 / 4)

# display answer
print("New Equilibrium Temperature: ", T_eq_2, "K or", T_eq_2 - 273, "C")

### Questions 3.1: Climate Connection

1.  Does a reduction in the transmissivity, $\tau$, imply a decrease or increase in OLR?
2.  How does the new equilibrium temperature compare to that calculated previously? Why do you think this is?


### Coding Exercises 3.1

1. Plot the equilibrium temperature as a function of $\tau$, for $\tau$ ranging from zero to one.


In [ ]:
# define the observed insolation based on observations from the IPCC AR6 Figure 7.2
Q = ...

# define the observed reflected radiation based on observations from the IPCC AR6 Figure 7.2
F_ref = ...

# define albedo
alpha = ...

# define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
sigma = ...


# define a function that returns the equilibrium temperature and takes argument tau
def get_eqT(tau):
    return ...


# define tau as an array extending from 0 to 1 with spacing interval 0.01
tau = ...

# use list comprehension to obtain the equilibrium temperature as a function of tau
eqT = ...

fig, ax = plt.subplots()
# Plot tau vs. eqT
_ = ...
ax.set_xlabel(...)
ax.set_ylabel(...)

# Summary
In this tutorial, you explored the fundamentals of Earth's energy balance.  You learned how to calculate Earth's albedo $\mathbf{\alpha}$ and how absorbed shortwave radiation contributes to energy balance. You also discovered the concept of equilibrium temperature and it's relationship to energy balance. The tutorial also highlighted the impact of increasing greenhouse gases on the equilibrium temperature.